In [7]:
import datetime#import datetime module convert data to datetime64(ns)
import numpy as np
import pandas as pd#import pandas module(Better than csv module!)
import os
import glob

global train_x , train_y,TimeData,Type
train_x=[]
train_y=[]

def train_norm(x):
        return (x-np.mean(x))/(np.max(x)-np.min(x))

def GenerateTrain(Type,feature):
    for tmp in TimeData:
            #print(tmp[1])
            i=tmp[1].dropna()
            IsWeekOrNot=int(i.size)/7
            if IsWeekOrNot==5:
                result=i.loc[ : ,feature].values.tolist() 
                train_x.append(i.loc[ : ,feature].values.tolist())
                Open=i.loc[:,'open'].values.tolist()[0]
                Close=i.loc[:,Type].values.tolist()[len(i.loc[:,Type].values.tolist())-1]
                train_y.append(float(Close-Open))
            else:
                continue
    nptrain=np.array(train_x)
    #nptrain = train_norm(nptrain)
    np.save(os.path.join('./StockData/TrainingData/','trainingX_'+path[12:21]),nptrain)
    print(nptrain)
    print(path[12:21],nptrain.shape)
    nptrain=np.array(train_y)
    np.save(os.path.join('./StockData/TrainingData/','trainingY_'+path[12:21]),nptrain)
    print(path[12:21],nptrain.shape)


tmp=input("Enter the needed features seperated with space: ")
feature=tmp.split(" ")
closeType=input("Enter the close type: ")
#print(feature)
resam=input("Enter the desired  data span , d  for day/ w for week / m for month: ")
#print(resam)
for path in glob.glob(r'./StockData/stock0050.csv'):
    CsvData=pd.DataFrame(pd.read_csv(path))
    CsvData['date']=pd.to_datetime(CsvData['date'])
    CsvData["week"]=0
    print(CsvData)
    for i,j in zip(CsvData["date"],range(CsvData["date"].size)):
        CsvData.loc[j:j+1,"week"]=i.isocalendar()[2]
    CsvData=CsvData.drop([0],axis=0)#drop 第一天 因為stockdata 有16年跳到17年的問題
    print(CsvData)
    TimeData=CsvData.set_index('date').resample(resam)
    GenerateTrain(closeType,feature)
    train_x.clear()
    train_y.clear()

Enter the needed features seperated with space: low high
Enter the close type: close
Enter the desired  data span , d  for day/ w for week / m for month: w
           date  adj_close  close   high    low   open   volume  week
0    2004-02-11      48.30  48.37  48.50  48.26  48.50  5264000     0
1    2004-02-12      48.54  48.59  49.26  48.40  48.50  8114000     0
2    2004-02-13      49.41  49.46  49.47  48.60  48.61  6138000     0
3    2004-02-16      49.17  49.20  49.46  49.04  49.46  4154000     0
4    2004-02-17      49.40  49.50  49.50  49.16  49.21  4645000     0
...         ...        ...    ...    ...    ...    ...      ...   ...
3912 2019-12-05      92.85  92.90  93.00  92.20  92.35  3441050     0
3913 2019-12-06      92.95  93.05  93.50  92.75  93.30  3394433     0
3914 2019-12-09      93.60  93.65  93.80  93.20  93.20  2912194     0
3915 2019-12-10      93.00  93.00  93.40  92.80  93.40  5465431     0
3916 2019-12-11      93.75  93.75  93.75  92.95  93.00  1936772     0

[39

## 